># 유통 판매량 예측 및 재고 최적화

# 단계 3 : 모델링 및 비즈니스 평가

# 라이브러리 호출

In [ ]:
import utils.dataloader
import utils.preprocessing
import utils.dataloader
import utils.train
import utils.test
import utils.evaluate

import matplotlib.pyplot as plt
from sklearn.metrics import *
import pickle
import warnings
warnings.filterwarnings('ignore')

# 모델링

In [ ]:
# Data Load
train_path = "./data/train"
datas = utils.dataloader.train_loader(train_path)

# Training Step
model_type = 'transformer' # lstm, cnn, transformer, conv_lstm
store_id = 44
product_id = 3
data_info = utils.preprocessing.preprocess(datas, model_type, store_id=store_id, product_id=product_id,
                                           timestep=24, training=True,
                                           x_scaling=True, y_scaling=False)

model_params = {
    "epochs" : 100,
    "batch_size" : 32,
    "valid_rate" : 0.2
}
name = "_".join((model_type, str(store_id), str(product_id))) + ".onnx"
model_info = utils.train.trainer(data_info, model_type, model_params, name)

# Test

In [ ]:

# Test Step
test_path = "./data/test"
test_data = datas = utils.dataloader.test_loader(test_path)
price = test_data["price"]

model_type = 'transformer' # lstm, cnn, transformer, conv_lstm
store_id = 44
product_id = 3
test_data_info = utils.preprocessing.preprocess(test_data, model_type, store_id=store_id, product_id=product_id,
                                           timestep=24, training=False,
                                           x_scaling=True, y_scaling=False)

model_onnx = f"./{model_type}_{store_id}_{product_id}.onnx"
y_pred = utils.test.predictor(test_data_info["X"], model_onnx)[0]

with open('./y_scaler.pkl', 'rb') as file:
    y_scaler = pickle.load(file)
    
y_true = y_scaler.inverse_transform(test_data_info["y"])
y_pred = y_scaler.inverse_transform(y_pred)


mae = mean_absolute_error(y_true, y_pred)
rmse = mean_squared_error(y_true, y_pred, squared=False)
r2 = r2_score(y_true, y_pred)
print(f"mae : {mae}\nmse : {rmse}\nr2 score : {r2}")
print()
inventory = utils.evaluate.inv_simulator(y_true, y_pred, 30, price)


plt.figure(figsize = (8,5))
plt.plot(y_true, label = 'Y true')
plt.plot(y_pred, label = 'Y pred')
plt.legend()
plt.grid()